In [1]:
%matplotlib inline
from edward.util import Progbar
import tensorflow as tf
from utils import generator_xy
import keras
import edward as ed
import numpy as np
from observations import mnist
import matplotlib.pyplot as plt
from edward.models import Normal, Bernoulli, Categorical
import pickle
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
def load_data(path):
    f = open(path,'rb')
    return pickle.load(f)

In [3]:
x_train_AE = load_data('./data/encoded_train_AE.pkl')
x_test_AE = load_data('./data/encoded_test_AE.pkl')
x_train_BAE = load_data('./data/encoded_train_BAE.pkl')
x_test_BAE = load_data('./data/encoded_test_BAE.pkl')
(_, y_train), (_, y_test) = mnist('./data')

In [4]:
M = 10000
x_train_generator = generator_xy([x_train_AE, y_train], M)

In [5]:
sess = tf.InteractiveSession()
def NN_classifier(x, W_0, W_1, b_0, b_1):
    h = tf.nn.relu(tf.matmul(x,W_0) + b_0)
    h = tf.matmul(h,W_1) + b_1
    h = tf.nn.softmax(h)
    return h

In [6]:
W_0 = Normal(loc = tf.zeros([256, 100]), scale = tf.ones([256, 100]))
W_1 = Normal(loc = tf.zeros([100, 10]), scale = tf.ones([100,10]))
b_0 = Normal(loc = tf.zeros(100), scale = tf.ones(100))
b_1 = Normal(loc = tf.zeros(10), scale = tf.ones(10))

x = tf.placeholder(tf.float32, [None, 256])

y = Categorical(logits=NN_classifier(x, W_0, W_1, b_0, b_1))


In [7]:
qW_0 = Normal(loc = tf.Variable(tf.random_normal([256,100])),
                  scale = tf.nn.softplus(tf.Variable(tf.random_normal([256,100]))))
qW_1 = Normal(loc = tf.Variable(tf.random_normal([100,10])),
                  scale = tf.nn.softplus(tf.Variable(tf.random_normal([100,10]))))
qb_0 = Normal(loc = tf.Variable(tf.random_normal([100])),
                  scale = tf.nn.softplus(tf.Variable(tf.random_normal([100]))))
qb_1 = Normal(loc = tf.Variable(tf.random_normal([10])),
                   scale = tf.nn.softplus(tf.Variable(tf.random_normal([10]))))


In [8]:
y_train = tf.cast(y_train, tf.int32)
y_test = tf.cast(y_test, tf.int32)
inference = ed.KLqp({W_0:qW_0, b_0:qb_0,
                    W_1:qW_1, b_1:qb_1}, data = {x: x_train_AE, y: y_train})
inference.initialize(n_iter = 1000, n_samples = 5)
tf.global_variables_initializer().run()

In [9]:
n_epoch = 100
n_iter_per_epoch = x_train_AE.shape[0] // M
log_loss = []
for epoch in range(1, n_epoch + 1):
  print("Epoch: {0}".format(epoch))
  avg_loss = 0.0

  pbar = Progbar(n_iter_per_epoch)
  for t in range(1, n_iter_per_epoch + 1):
    pbar.update(t)
    x_batch, y_batch = next(x_train_generator)
    info_dict = inference.update(feed_dict={x: x_batch, y: y_batch})
    avg_loss += info_dict['loss']
  avg_loss = avg_loss / n_iter_per_epoch
  avg_loss = avg_loss / M
  log_loss.append(-avg_loss)
  print("-log p(x) <= {:0.3f}".format(avg_loss))

Epoch: 1
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 16.088
Epoch: 2
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 15.058
Epoch: 3
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 14.301
Epoch: 4
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 14.085
Epoch: 5
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 13.665
Epoch: 6
6/6 [100%] ██████████████████████████████ Elapsed: 4s
-log p(x) <= 13.362
Epoch: 7
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 13.153
Epoch: 8
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 12.937
Epoch: 9
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 12.699
Epoch: 10
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 12.506
Epoch: 11
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 12.374
Epoch: 12
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 12.203
E

6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 10.429
Epoch: 99
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 10.446
Epoch: 100
6/6 [100%] ██████████████████████████████ Elapsed: 3s
-log p(x) <= 10.464


In [29]:
y_pred = NN_classifier(x_train_AE, qW_0.sample(),qW_1.sample(),qb_0.sample(), qb_1.sample())
y_pred = [np.argmax(i) for i in y_pred.eval()]

correct = 0
for i in range(len(y_train.eval())):
    if y_train.eval()[i] == y_pred[i]:
        correct += 1
    else:
        pass
print(correct/len(y_train.eval())* 1.)

0.79515
